# Military Drone Strikes, Afgan, Pakistan, Yemen and Somalia

This script is used to prepare our data for future analysis. it reads in the DroneWarsData_tw.xlsx spreadsheet and outputs drone-strikes.xlsx spreadsheet.

We can then use the data in drone-strikes.xlsx in our data analysis piece

The DroneWarsData.xlsx can be downloaded from the dronewars.io website
https://dronewars.github.io/data/DroneWarsData.xlsx

In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import mitosheet
import re

In [2]:
# Read in the 'All' sheet
df = pd.read_excel('DroneWarsData.xlsx', sheet_name = 'All')

In [3]:
df.head()

,Strike ID,Country,Date (MM-DD-YYYY),President,Most Specific Location,Most Specific Lat/Long,Latitude,Longitude,Minimum total people killed,Maximum total people killed,Minimum civilians reported killed,Maximum civilians reported killed,Minimum children reported killed,Maximum children reported killed,Minimum reported injured,Ratio of Civilians to Total Killed,Other (Non-civilian/children Killed)
0,B50,Pakistan,01-01-2009,Bush,"Kari Kot, South Waziristan, Pakistan","32.270651,69.554179",32.270651,69.554179,3,5,0,0,0,0,1,0.0,5
1,Ob53,Pakistan,01-01-2010,Obama,"Ghundikala, North Waziristan, Pakistan","32.959854,70.156371",32.959854,70.156371,2,3,0,0,0,0,3,0.0,3
2,Ob181,Pakistan,01-01-2011,Obama,"Mandi Khel, North Waziristan, Pakistan","33.001031,70.364339",33.001031,70.364339,8,10,0,0,0,0,0,0.0,10
3,Ob182,Pakistan,01-01-2011,Obama,"Mandi Khel, North Waziristan, Pakistan","33.001031,70.364339",33.001031,70.364339,4,6,0,0,0,0,0,0.0,6
4,Ob183,Pakistan,01-01-2011,Obama,"Datta Khel, North Waziristan, Pakistan","33.150049,70.433361",33.150049,70.433361,4,5,0,0,0,0,0,0.0,5


In [4]:
# Rename our date column to Date
df.rename(columns = {'Date (MM-DD-YYYY)' : 'Date'}, inplace=True)

In [5]:
# convert our date to a pandas datetime field
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
# Add a Year and Month column to our dataframe
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] =pd.DatetimeIndex(df['Date']).month
df['Day'] = df['Date'].dt.day_name()

## Lets see what we have

In [7]:
# Check we loaded our data
df.head()

,Strike ID,Country,Date,President,Most Specific Location,Most Specific Lat/Long,Latitude,Longitude,Minimum total people killed,Maximum total people killed,Minimum civilians reported killed,Maximum civilians reported killed,Minimum children reported killed,Maximum children reported killed,Minimum reported injured,Ratio of Civilians to Total Killed,Other (Non-civilian/children Killed),Year,Month,Day
0,B50,Pakistan,2009-01-01,Bush,"Kari Kot, South Waziristan, Pakistan","32.270651,69.554179",32.270651,69.554179,3,5,0,0,0,0,1,0.0,5,2009,1,Thursday
1,Ob53,Pakistan,2010-01-01,Obama,"Ghundikala, North Waziristan, Pakistan","32.959854,70.156371",32.959854,70.156371,2,3,0,0,0,0,3,0.0,3,2010,1,Friday
2,Ob181,Pakistan,2011-01-01,Obama,"Mandi Khel, North Waziristan, Pakistan","33.001031,70.364339",33.001031,70.364339,8,10,0,0,0,0,0,0.0,10,2011,1,Saturday
3,Ob182,Pakistan,2011-01-01,Obama,"Mandi Khel, North Waziristan, Pakistan","33.001031,70.364339",33.001031,70.364339,4,6,0,0,0,0,0,0.0,6,2011,1,Saturday
4,Ob183,Pakistan,2011-01-01,Obama,"Datta Khel, North Waziristan, Pakistan","33.150049,70.433361",33.150049,70.433361,4,5,0,0,0,0,0,0.0,5,2011,1,Saturday


In [8]:
mitosheet.sheet(df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-81ed1192-245e-4a19-9625-4d4ede2e339f", "code": {"imports…

In [9]:
# Lets see what we have
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Strike ID                             1696 non-null   object        
 1   Country                               1696 non-null   object        
 2   Date                                  1696 non-null   datetime64[ns]
 3   President                             1696 non-null   object        
 4   Most Specific Location                1696 non-null   object        
 5   Most Specific Lat/Long                1696 non-null   object        
 6   Latitude                              1659 non-null   float64       
 7   Longitude                             1660 non-null   float64       
 8   Minimum total people killed           1696 non-null   object        
 9   Maximum total people killed           1696 non-null   object        
 10  

In [10]:
# Check our Column Names
df.columns

Index(['Strike ID', 'Country', 'Date', 'President', 'Most Specific Location',
       'Most Specific Lat/Long', 'Latitude', 'Longitude',
       'Minimum total people killed', 'Maximum total people killed',
       'Minimum civilians reported killed',
       'Maximum civilians reported killed', 'Minimum children reported killed',
       'Maximum children reported killed', 'Minimum reported injured',
       'Ratio of Civilians to Total Killed',
       'Other (Non-civilian/children Killed)', 'Year', 'Month', 'Day'],
      dtype='object')

## Data Cleansing

### Trim Trailing spaces from columns in Pandas

In [11]:
# Trim leading and trailing spaces
df['Country'] = df['Country'].str.strip()
df['Most Specific Location'] = df['Most Specific Location'].str.strip()

### Dealing with some of our 'Unknown' values
This code deals with data in our 'Country' column that is set to 'Unknown".
We fix this so when we move on to the next section when we clean up Latitude
and Longitude we have valid Country names.

In [12]:
# Data Clensing
# Fix Unknown 'Most Specific Location'
# Set our Country Masks
is_yemen = df['Country'] == 'Yemen'
is_somalia = df['Country'] == 'Somalia'
is_pakistan = df['Country'] == 'Pakistan'
is_afghanistan =df['Country'] == 'Afghanistan'

# Set a mask to grab our Unknown 'Most Specific Locations'
# We will use this to set the 'Most Specific Location'
is_unknown = df['Most Specific Location'] == 'Unknown'

# Fix the spelling of Afghanistan
is_afganistan = df['Country'] == 'Afganistan'
df.loc[(is_afganistan), 'Country'] = 'Afghanistan'

# Fix up our Unknown 'Most Specific Locations' by country
df.loc[(is_yemen & is_unknown), 'Most Specific Location'] = 'Yemen, Unknown'
df.loc[(is_somalia & is_unknown), 'Most Specific Location'] = 'Somalia, Unknown'
df.loc[(is_pakistan & is_unknown), 'Most Specific Location'] = 'Pakistan, Unknown'
df.loc[(is_afghanistan & is_unknown), 'Most Specific Location'] = 'Afghanistan, Unknown'

### Setting Lat and Long for our previously 'Unknown' locations
Here I set the Lat and Long for any 'Unknown' detected in the step above. I have used a fefault Lat/Long pair for each Country. This was just chosen at random from Google Maps

In [13]:
# Get a list off all the rows where there is an unknown for "Most Specific Lat/Long"
ms_latlong = df["Most Specific Lat/Long"] == 'Unknown'

# Using our Country mask from the step above and our new ms_latlong mask we can assign
# default lat long co-ords to any data in the'Most Specific Lat/Long' column
df.loc[(is_yemen & ms_latlong), "Most Specific Lat/Long"] = '14.454376, 45.102241'
df.loc[(is_somalia & ms_latlong), "Most Specific Lat/Long"] = '-1.496491, 41.592160'
df.loc[(is_pakistan & ms_latlong), "Most Specific Lat/Long"] = '33.976996, 69.951233'
df.loc[(is_afghanistan & ms_latlong), "Most Specific Lat/Long"] = '29.484804, 62.761735'

### Dealing with a particular Excel error in our dataset
Because the dataset has used data from calculated fields in our initial spreadsheet,  some of these claculated fields were not valid and Excel marks them as '#VALUE!' error. When we import this data into pandas the errors come across.

In [14]:
# Fix '#VALUE!'
is_value = df['Other (Non-civilian/children Killed)'] == '#VALUE!'
df.loc[(is_value), 'Other (Non-civilian/children Killed)'] = 0

### Replacing one data value for another in specific columns

In [15]:
# Replace the minus sign in the column list below
col = ['Minimum total people killed', 'Maximum total people killed',
       'Minimum civilians reported killed','Maximum civilians reported killed',
       'Minimum children reported killed', 'Maximum children reported killed',
       'Minimum reported injured' ]

for i in col:
  df[i] = df[i].replace('-', "0")

### Fix the erroneous data in the 'Most Specific Lat/Long'
Because humans added this data somewhere they have also intriduced errors. 
Using a regular expresion we can clean up our data in this column efficently

In [16]:
# Create list to hold our lLatitude and Longitude cleaned data
lat_list = []
long_list = []

# Set the Regular Expresion that we want to find in our data
pattern = re.compile(r"-?\d\d?\.\d+")

# Loop through our column and extract the Lat and long values and append to our lists
for pair in df['Most Specific Lat/Long']:
    match = pattern.findall(pair)
    if len(match) == 2:
        lat, long = match
        lat_list.append(lat)
        long_list.append(long)

# Set the 'Latitude' and 'Longitude' values to our cleaned values
df['Latitude'] = lat_list
df['Longitude'] = long_list

### Set our column datatypes

In [17]:
# Sort out our data types
df = df.astype({'Latitude': 'float64',
                'Longitude' : 'float64',
                'Minimum total people killed': 'int64',
                'Maximum total people killed': 'int64',
                'Minimum civilians reported killed': 'int64',
                'Maximum civilians reported killed': 'int64',
                'Minimum children reported killed': 'int64',
                'Maximum children reported killed': 'int64',
                'Minimum reported injured': 'int64',
                'Other (Non-civilian/children Killed)': 'int64'
               })

In [18]:
mitosheet.sheet(df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-80d53523-2b13-4aad-91fa-afec1b1bc1f9", "code": {"imports…

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Strike ID                             1696 non-null   object        
 1   Country                               1696 non-null   object        
 2   Date                                  1696 non-null   datetime64[ns]
 3   President                             1696 non-null   object        
 4   Most Specific Location                1696 non-null   object        
 5   Most Specific Lat/Long                1696 non-null   object        
 6   Latitude                              1696 non-null   float64       
 7   Longitude                             1696 non-null   float64       
 8   Minimum total people killed           1696 non-null   int64         
 9   Maximum total people killed           1696 non-null   int64         
 10  

### Save our cleansed data to a new spreadsheet

In [20]:
# Export our newly formated dataframe to excel but drop our pandas index column
df.to_excel('C:/Users/twelsh/data/Notebooks/drone-strikes/published/drone-strikes.xlsx', index=False)